In [ ]:
!pip install scikit-learn

In [ ]:
!pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 2.5 MB/s eta 0:00:00


In [64]:
#Importar bibliotecas
from sklearn.datasets import load_iris
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [53]:
#Cargar el conjunto de datos iris
iris = load_iris()
iris_df = pd.DataFrame(data=iris['data'], columns=iris['feature_names'])
iris_df['target'] = iris['target']

In [54]:
#Mostrar las primeras filas del conjunto de datos
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [55]:
#tecnica de super vector machine
#Dividir el conjunto de datos en entrenamiento y prueba
x = iris_df.drop('target', axis=1)
y = iris_df['target']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

#Definir el modelo SVM
svm_model = SVC()

#Mostrar los hiperparámetros por defecto del modelo
svm_model.get_params(deep=True)


{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [56]:
#GridSearch
#Definir la cuadricula de hiperparámetros
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear','rbf'],
    'gamma': ['scale','auto']
}

#Crear el objeto GridSearchCV
grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=5)

#Ajustar el modelo utilizando la búsqueda en Grid
grid_search.fit(x_train, y_train)

#Obtener los mejores hiperparámetros encontrados
best_params_grid_search = grid_search.best_params_
best_params_grid_search

{'C': 1, 'gamma': 'scale', 'kernel': 'linear'}

In [57]:
#RandomizedSearchCV

#Definir la distribución de hiperparámetros
param_dist = {
    'C': uniform(0.1,100),
    'kernel': ['linear','rbf'],
    'gamma': ['scale','auto']
}

#Crear el objeto RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=svm_model, param_distributions=param_dist, n_iter=100, cv=5, random_state=42)

#Ajustar el modelo utilizando la búsqueda aleatoria
random_search.fit(x_train, y_train)

#Obtener los mejores parámetros encontrados
best_params_random_search = random_search.best_params_
best_params_random_search

{'C': 3.5388521115218396, 'gamma': 'auto', 'kernel': 'linear'}

In [60]:
#Optimización Bayesiana
#Definir un espacio de búsqueda reducido para la optimización bayesiana
param_space_reduce = {
    'C': (0.1, 5),
    'kernel': ['linear','rbf'],
    'gamma': ['scale','auto']
}

#Crear el objeto BayesSearchDV con un número reducido de iteraciones
bayes_search_reduced = BayesSearchCV(estimator=svm_model, search_spaces=param_space_reduce, n_iter=25, cv=5, random_state=42)

#Ajustar el modelo utilizando la búsqueda aleatoria
bayes_search_reduced.fit(x_train, y_train)

#Obtener los mejores parámetros encontrados
best_params_random_search = bayes_search_reduced.best_params_
best_params_random_search

OrderedDict([('C', 4.203202942211093),
             ('gamma', 'auto'),
             ('kernel', 'linear')])

In [63]:
#Evaluar modelo obtenido con la búsqueda  en Grid
grid_search_model = grid_search.best_estimator_
grid_search_predictions = grid_search_model.predict(x_test)
grid_search_accuracy = accuracy_score(y_test, grid_search_predictions)

#Evaluar el modelo obtenido en la búsqueda aleatoria
random_search_model = random_search.best_estimator_
random_search_preditcions = random_search_model.predict(x_test)
random_search_accuracy = accuracy_score(y_test, random_search_preditcions)

#Evaluar el model obtenido con la optimización bayesiana
bayes_search_model = bayes_search_reduced.best_estimator_
bayes_search_predictions = bayes_search_model.predict(x_test)
bayes_search_accuracy = accuracy_score(y_test, bayes_search_predictions)


grid_search_accuracy, random_search_accuracy, bayes_search_accuracy

#El mejor resultado es por búsqueda aleatoria

(1.0, 0.9666666666666667, 0.9666666666666667)

In [65]:
#Validacion Cruzada en Grid

#Utilizar el mejor modelo encontrado por la búsqueda en Grid
best_model = grid_search.best_estimator_

#Realizar una validación cruzada de 5 pliegues en el conjunto de datos completo (x,y)
cv_scores = cross_val_score(best_model, x, y, cv=5)

#Calcular la media y la desviación estándar de los puntajes de la validación cruzada
cv_mean = cv_scores.mean()
cv_std = cv_scores.std()

cv_mean, cv_std

(0.9800000000000001, 0.016329931618554516)

In [66]:
#Validacion Cruzada en Búsqueda aleatoria y optimizacion bayesiana

#Utilizar el mejor modelo encontrado por la búsqueda Aleatoria
best_model_random_search =random_search.best_estimator_

#Utilizar el mejor modelo encontrado por la Optimización Bayesiana
best_model_bayes_search =bayes_search_reduced.best_estimator_

#Realizar una validación cruzada de 5 pliegues en el conjunto de datos completo (x,y)
cv_scores_grid_search = cross_val_score(best_model, x, y, cv=5)
cv_scores_random_search = cross_val_score(best_model_random_search, x, y, cv=5)
cv_scores_bayes_search = cross_val_score(best_model_bayes_search, x, y, cv=5)

#Calcular la media y la desviación estándar de los puntajes de la validación cruzada
cv_mean_grid_search, cv_std_grid_search = cv_scores_grid_search.mean(), cv_scores_grid_search.std()
cv_mean_random_search, cv_std_random_search = cv_scores_random_search.mean(), cv_scores_random_search.std()
cv_mean_bayes_search, cv_std_bayes_search = cv_scores_bayes_search.mean(), cv_scores_bayes_search.std()

cv_mean_grid_search, cv_std_grid_search, cv_mean_random_search, cv_std_random_search, cv_mean_bayes_search, cv_std_bayes_search

(0.9800000000000001,
 0.016329931618554516,
 0.9733333333333334,
 0.02494438257849294,
 0.9733333333333334,
 0.02494438257849294)